<a href="https://colab.research.google.com/github/Sattu879/denoising_vlg/blob/main/denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

from google.colab import drive
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Navigate to the current directory containing the notebook
notebook_directory = '/content/drive/My Drive/Colab Notebooks'

os.chdir(notebook_directory)

# Verify the current directory
print("Current directory:", os.getcwd())

# Step 3: Define the directory path
test_directory = 'Train'
os.chdir(test_directory)

# Verify the current directory
print("Current directory:", os.getcwd())

# Step 4: Define the paths to the high and low quality directories
high_quality_dir = 'high'
low_quality_dir = 'low'

# Verify the directories exist
if os.path.exists(high_quality_dir) and os.path.exists(low_quality_dir):
    print(f"'high' directory is located at: {os.path.abspath(high_quality_dir)}")
    print(f"'low' directory is located at: {os.path.abspath(low_quality_dir)}")
else:
    print("One or both of the directories do not exist. Please check the paths.")


Mounted at /content/drive
Current directory: /content/drive/My Drive/Colab Notebooks
Current directory: /content/drive/My Drive/Colab Notebooks/Train
One or both of the directories do not exist. Please check the paths.


In [2]:
high_quality_images = sorted(glob.glob(os.path.join(high_quality_dir, '*.png')))
low_quality_images = sorted(glob.glob(os.path.join(low_quality_dir, '*.png')))

assert len(high_quality_images) == len(low_quality_images), "The number of images in both directories must be the same."


print(f"Found {len(high_quality_images)} high-quality images.")
print(f"Found {len(low_quality_images)} low-quality images.")

# Function to load and pair images
def load_image_pairs(high_quality_images, low_quality_images):
    pairs = []
    for hq_img_path, lq_img_path in zip(high_quality_images, low_quality_images):
        hq_img = cv2.imread(hq_img_path, cv2.IMREAD_COLOR)
        lq_img = cv2.imread(lq_img_path, cv2.IMREAD_COLOR)
        if hq_img is None or lq_img is None:
            print(f"Error reading images: {hq_img_path}, {lq_img_path}")
        else:
            pairs.append((hq_img, lq_img))
    return pairs

image_pairs = load_image_pairs(high_quality_images, low_quality_images)

if len(image_pairs) == 0:
    print("No image pairs were loaded. Check your file paths and image formats.")
else:
    print(f"Successfully loaded {len(image_pairs)} image pairs.")

Found 0 high-quality images.
Found 0 low-quality images.
No image pairs were loaded. Check your file paths and image formats.


In [3]:
# Normalization
def normalize_image(image):
    return image.astype(np.float32) / 255.0

# Normalize all pairs
normalized_pairs = [(normalize_image(hq), normalize_image(lq)) for hq, lq in image_pairs]

print(f"First normalized pair shapes: {normalized_pairs[0][0].shape}, {normalized_pairs[0][1].shape}")

high_quality_images, low_quality_images = zip(*normalized_pairs)

hq_train, hq_test, lq_train, lq_test = train_test_split(
    high_quality_images, low_quality_images, test_size=0.2, random_state=42)

hq_train, hq_val, lq_train, lq_val = train_test_split(
    hq_train, lq_train, test_size=0.25, random_state=42)


hq_train, hq_val, hq_test = np.array(hq_train), np.array(hq_val), np.array(hq_test)
lq_train, lq_val, lq_test = np.array(lq_train), np.array(lq_val), np.array(lq_test)

print("Training set size:", len(hq_train))
print("Validation set size:", len(hq_val))
print("Test set size:", len(hq_test))

IndexError: list index out of range

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class ImagePairDataset(Dataset):
    def __init__(self, hq_images, lq_images):
        self.hq_images = hq_images
        self.lq_images = lq_images

    def __len__(self):
        return len(self.hq_images)

    def __getitem__(self, idx):
        hq_img = self.hq_images[idx]
        lq_img = self.lq_images[idx]
        hq_img = torch.from_numpy(hq_img.transpose((2, 0, 1)))
        lq_img = torch.from_numpy(lq_img.transpose((2, 0, 1)))
        return hq_img, lq_img

# Create data loaders
batch_size = 16
train_dataset = ImagePairDataset(hq_train, lq_train)
val_dataset = ImagePairDataset(hq_val, lq_val)
test_dataset = ImagePairDataset(hq_test, lq_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


NameError: name 'hq_train' is not defined

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim

class DenoisingCNN(nn.Module):
    def __init__(self):
        super(DenoisingCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [ ]:
def initialize_model():
    model = DenoisingCNN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return model, criterion, optimizer

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImagePairDataset(Dataset):
    def __init__(self, hq_images, lq_images):
        self.hq_images = hq_images
        self.lq_images = lq_images

    def __len__(self):
        return len(self.hq_images)

    def __getitem__(self, idx):
        hq_img = self.hq_images[idx]
        lq_img = self.lq_images[idx]
        hq_img = torch.from_numpy(hq_img.transpose((2, 0, 1))).float()
        lq_img = torch.from_numpy(lq_img.transpose((2, 0, 1))).float()
        return hq_img, lq_img

def create_dataloaders(hq_train, lq_train, hq_val, lq_val, batch_size=16):
    train_dataset = ImagePairDataset(hq_train, lq_train)
    val_dataset = ImagePairDataset(hq_val, lq_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            print('It is working')
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs_val, targets_val in val_loader:
                inputs_val, targets_val = inputs_val.to(device), targets_val.to(device)
                outputs_val = model(inputs_val)
                loss = criterion(outputs_val, targets_val)
                val_loss += loss.item() * inputs_val.size(0)

        val_loss /= len(val_loader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    return model

In [ ]:

batch_size = 16
train_loader, val_loader = create_dataloaders(hq_train, lq_train, hq_val, lq_val, batch_size=batch_size)

model, criterion, optimizer = initialize_model()

# Train the model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=2)

# Save the model
torch.save(trained_model.state_dict(), '/content/denoising_model.pth')


In [ ]:
# Load the saved model
model = DenoisingCNN()
model.load_state_dict(torch.load('/content/denoising_model.pth'))
model.eval()

# Evaluate on test set
test_loss = 0.0
with torch.no_grad():
    for inputs_test, targets_test in test_loader:
        outputs_test = model(inputs_test.float())
        loss = criterion(outputs_test, targets_test.float())
        test_loss += loss.item() * inputs_test.size(0)

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')



In [ ]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def calculate_metrics(denoised, original):
    psnr_value = peak_signal_noise_ratio(original, denoised)
    ssim_value, _ = structural_similarity(original, denoised, win_size=5, full=True, multichannel=True)
    return psnr_value, ssim_value

psnr_values = []
ssim_values = []

with torch.no_grad():
    for idx, (lq_tensor, original_img) in enumerate(test_loader):
        denoised_tensor = model(lq_tensor.float())
        denoised_img = denoised_tensor.squeeze().cpu().numpy()
        original_img = original_img.squeeze().cpu().numpy()


In [ ]:
import skimage.transform

denoised_img = skimage.transform.resize(denoised_img, (500, 700))
original_img = skimage.transform.resize(original_img, (500, 700))


In [ ]:
psnr_value, ssim_value = calculate_metrics(denoised_img, original_img)